In [6]:

import numpy as np
from collections import Counter


data: (N, n_features +1) (last columns are the labels)

In [7]:

from sklearn.datasets import load_iris

In [18]:
X, y = load_iris(return_X_y=True)
data = np.hstack((X,y.reshape(-1, 1)))
np.random.shuffle(data)

In [29]:
data[data[:,0] <5,:]

array([[4.4, 2.9, 1.4, 0.2, 0. ],
       [4.6, 3.1, 1.5, 0.2, 0. ],
       [4.9, 3.1, 1.5, 0.2, 0. ],
       [4.8, 3.1, 1.6, 0.2, 0. ],
       [4.9, 2.4, 3.3, 1. , 1. ],
       [4.9, 3.6, 1.4, 0.1, 0. ],
       [4.4, 3. , 1.3, 0.2, 0. ],
       [4.6, 3.4, 1.4, 0.3, 0. ],
       [4.3, 3. , 1.1, 0.1, 0. ],
       [4.8, 3.4, 1.6, 0.2, 0. ],
       [4.4, 3.2, 1.3, 0.2, 0. ],
       [4.6, 3.2, 1.4, 0.2, 0. ],
       [4.8, 3. , 1.4, 0.1, 0. ],
       [4.9, 3.1, 1.5, 0.1, 0. ],
       [4.9, 3. , 1.4, 0.2, 0. ],
       [4.6, 3.6, 1. , 0.2, 0. ],
       [4.7, 3.2, 1.3, 0.2, 0. ],
       [4.8, 3.4, 1.9, 0.2, 0. ],
       [4.5, 2.3, 1.3, 0.3, 0. ],
       [4.9, 2.5, 4.5, 1.7, 2. ],
       [4.8, 3. , 1.4, 0.3, 0. ],
       [4.7, 3.2, 1.6, 0.2, 0. ]])

In [23]:

class Question:
    def __init__(self, feature, threshold):
        self.feature = feature
        self.threshold = threshold
    
    def answer(self, x):
        if x[self.feature] < self.threshold: return True
        return False
    
    def partition(self, rows):
        left_data, right_data = [], []
        for row in rows:
            if self.answer(row):
                left_data.append(row)
            else:
                right_data.append(row)
        return left_data, right_data



In [24]:


class DecisionNode:
    def __init__(self, question):
        self.question = question
        self.left = None
        self.right = None


def compute_frequencies(labels):
    freq = Counter(labels)
    for label, occurrence in freq.items():
        freq[label] = occurrence / len(labels)
    return freq

class LeafNode:
    def __init__(self, labels):
        self.probabilities = compute_frequencies(labels)


def gini_loss(data):
    frequencies = compute_frequencies(data[:,-1])
    gini = 0
    for frequency in frequencies.values():
        gini += frequency * (1 - frequency)
    return gini


def entropy_loss(data):
    frequencies = compute_frequencies(data[:,-1])
    entropy = 0
    for frequency in frequencies.values():
        entropy -= frequency * np.log(frequency)
    return entropy

#%%
class DecisionTreeClassifier:
    def __init__(self, loss=gini_loss):
        self.root = None
        self.loss = loss
    
    def fit(self, data):
        self.root = self.build_tree(data)

    def build_tree(self,data):
        gain, question = self.find_best_split(data)
        if gain == 0:
            return LeafNode(labels=data[:,-1])
        node = DecisionNode(question)
        left_data, right_data = question.partition(data) 
        node.left = self.build_tree(left_data)
        node.right = self.build_tree(right_data)
        return node

    def find_best_split(self, data):
        current_loss = self.loss(data)
        best_gain = 0
        best_question = None
        for feature in range(data.shape[1] - 1):
            unique_values = set(data[:, feature])
            for threshold in unique_values:
                question = Question(feature=feature, threshold=threshold)
                left_data, right_data = question.partition(data)
                split_gain = self.compute_split_gain(left_data, right_data, current_loss)
                if split_gain > best_gain:
                    best_gain, best_question = split_gain, question
                    

        return best_gain, best_question

    def compute_split_gain(self, left_data, right_data, current_loss):
        p = len(left_data) / (len(left_data) + len(right_data))
        left_loss = self.loss(left_data)
        right_loss = self.loss(right_data)
        return current_loss - (p * left_loss + (1-p) * right_loss)

    def predict_class(self, x):
        return self.predict_proba(x).most_common(1)[0]

    def predict_proba(self, x):
        """traverse tree + get counts of classes in the leaf"""
        node = self.root
        while not isinstance(node, LeafNode):
            node = node.decide(x)
        return node.probabilities




In [25]:
d = DecisionTreeClassifier()

In [26]:
d.fit(data)

TypeError: list indices must be integers or slices, not tuple